In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Rheumatoid_Arthritis/GSE176440'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiles of CD4+ T cells before and after methotrexate treatment in rheumatoid arthritis patients [Microarray]"
!Series_summary	"To understand the molecular mechanisms by which methotraxate improves the disease activity in rheumatoid arthritis, CD4+ T cells were obtained before and 3month after methotrexate treatment."
!Series_overall_design	"28 treatment naïve rheumatoid arthritis patients participated in the study. Blood samples were obtained before and 3 months after methotrexate treatment. CD4+ T cells were magnetically purified and subjected the DNA microarray analyses."
Sample Characteristics Dictionary:
{0: ['individual: A29', 'individual: A30', 'individual: A34', 'individual: C14', 'individual: C16', 'individual: C19', 'individual: C43', 'individual: C49', 'individual: C71', 'individual: C80', 'individual: C85', 'individual: C87', 'individual: C91', 'individual: C92', 'individual: C93', 'individual: C95', 'individual: C96',

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 1: Check for gene expression data availability
# Based on the series title and summary, this is a gene expression microarray study related to gene expression profiles.
is_gene_available = True

# Step 2: Variable availability and data type conversion

# Rheumatoid_Arthritis: The 'disease state: rheumatoid arthritis patient' in the sample characteristics dictionary suggests the presence of this variable.
# However, there's only one unique value 'rheumatoid arthritis patient', indicating no variability for associative studies.
trait_row = None

# Age: No information about age in the sample characteristics dictionary.
age_row = None

# Gender: No information about gender in the sample characteristics dictionary.
gender_row = None

# As trait_row is None, no clinical data extraction step is needed

# Save metadata
save_cohort_info('GSE176440', './preprocessed/Rheumatoid_Arthritis/cohort_info.json', is_gene_available, trait_row is not None)
